In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
from datetime import datetime, timedelta
import re

from scipy import sparse
from scipy.sparse.linalg import spsolve
import scipy.interpolate as interpolate
from scipy.signal import find_peaks

from pybaselines import Baseline, utils

from lmfit import Parameters, Minimizer
from lmfit.models import GaussianModel, ConstantModel, PolynomialModel, ExponentialModel, LinearModel

from ILL_IN8_functions.ill_data_processor import *

marker_list = ['.', ',', 'o', 'v', '^', '<', '>', '1', '2', '3', '4', 's', 'p', '*', 'h', 'H', '+', 'x', 'D', 'd', '|', '_'] 
color_list = [
    '#FF0000',  # Red
    '#00FF00',  # Green
    '#0000FF',  # Blue
    '#FFFF00',  # Yellow
    '#FF00FF',  # Magenta
    '#00FFFF',  # Cyan
    '#800000',  # Maroon
    '#008000',  # Dark Green
    '#000080',  # Navy
    '#808000',  # Olive
    '#800080',  # Purple
    '#008080',  # Teal
    '#FF4500',  # OrangeRed
    '#2E8B57',  # SeaGreen
    '#1E90FF',  # DodgerBlue
    '#FFD700',  # Gold
    '#FF69B4',  # HotPink
    '#8A2BE2',  # BlueViolet
    '#A52A2A',  # Brown
    '#DEB887'   # BurlyWood
]

In [3]:
# 현재 작업 디렉토리 불러오기
root_dir = os.getcwd()

# ILL 실험 데이터 디렉토리
data_dir = f'{root_dir}/Data/ILLData_242_in8_exp_7-01-619_rawdata'

# # .nxs 데이터는 가져오지 않기
file_name_list = sorted([file_name for file_name in os.listdir(data_dir) if not '.nxs' in file_name and not '.DS_Store' in file_name ])

# 여기에는 파일 리스트만 들어가면 된다
data_info_df = make_data_info_table(file_name_list, data_dir, save = False)

c:\Users\Byeongcheol Choe\OneDrive - Emergent Materials\4_Portfolio\ILL_IN8_Utilities\ILL_IN8_functions\ill_data_processor.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_info_df = pd.concat([data_info_df, pd.DataFrame(data_info_per_file, index=[0])], ignore_index=True)
c:\Users\Byeongcheol Choe\OneDrive - Emergent Materials\4_Portfolio\ILL_IN8_Utilities\ILL_IN8_functions\ill_data_processor.py:109: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  measure_delta_time = (pd.Series(pd.to_datetime(data_info_df['measure_date'].to_list()[1:]) -
c:\Users\Byeongcheol Choe\OneDrive - Emergent Materials\4_

In [4]:
data_info_df

,file_name,align/measure,H K L,energy_start,energy_end,energy_step,num_data_command,num_data_measured,mon,measure_date,measure_delta_time(min),instrument_setting_time(min),measure_time(min),command
0,049959,align,,,,,,,,24-May-24 11:24:58,4.7,4.3,0.4,sc a3 1.76 da3 0.5 np 21 ti 1
1,049960,align,,,,,,,,24-May-24 11:29:38,3.0,2.5,0.5,sc a3 53.42 da3 0.5 np 31 ti 1
2,049961,align,,,,,,,,24-May-24 11:32:35,1.1,1.0,0.1,bs a3 61 da3 0.5 np 21 ti 1
3,049962,align,,,,,,,,24-May-24 11:33:40,2.1,1.9,0.2,bs a3 61 da3 0.5 np 21 ti 1
4,049963,align,,,,,,,,24-May-24 11:35:47,7.0,6.8,0.2,sc gu 0 dgu 1 np 13 ti 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,051065,measure,-3.9 0.0 6.0,45.0,49.5,0.5,10,10,40000,27-Jun-24 08:02:08,8.1,2.2,5.9,bs qh -3.9 0.0 6.0 45 dqh 0 0 0 0.5 np 10 mn 4...
284,051066,measure,-4.0 0.0 6.0,45.0,49.5,0.5,10,10,40000,27-Jun-24 08:10:12,93.8,87.9,5.9,bs qh -4.0 0.0 6.0 45 dqh 0 0 0 0.5 np 10 mn 4...
285,051067,measure,15.0 0.0 0.0,70.0,80.0,0.5,21,3,50000,27-Jun-24 09:43:59,3.8,0.9,2.9,bs qh 15.0 0.0 0.0 70 dqh 0 0 0 0.5 np 21 mn 5...
286,051068,measure,15.0 0.0 0.0,65.0,75.0,0.5,21,18,50000,27-Jun-24 09:47:50,23.8,3.9,19.9,bs qh 15.0 0.0 0.0 65 dqh 0 0 0 0.5 np 21 mn 5...


# 1. 데이터 셋 인덱스 정리

In [ ]:
# low enervy 900_h
idx_list_900_h_10p0_30p0 = [39]
idx_list_900_h_30p5_40p0 = [45]
idx_list_900_h_30p5_40p5 = [51, 52, 53, 54, 55]

# low energy -204_h
idx_list_m204_h_10p0_30p0 = [41]
idx_list_m204_h_30p5_40p5 = [47]
idx_list_m204_h_10p0_40p0 = [56 ,47, 58, 59, 60]

# low energy 603_h
idx_list_603_h_10p0_30p0 = [44]
idx_list_603_h_30p0_40p5 = [50]
idx_list_603_h_10p0_40p0 = [61, 62, 63, 64, 65]

# high energy 1200_h
idx_list_1200_h_40p0_70p0 = [68, 69, 75, 76, 77, 78, 79, 80, 81, 82, 83] # 13 0 0에서 40 데이터 손실, 40.5 부터 측정
idx_list_1200_h_40p0_70p0_2 = [95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105] # 13 0 0 에서 데이터 40.5 부터 측정

# high energy 500_l
idx_list_005_l_40p0_70p0 = [73, 74, 86, 87, 88, 89, 90, 91, 92, 93, 94]
idx_list_005_l_45p0_65p0_2 = [133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143]

# high energy -306_h
idx_list_m306_h_40p0_70p0 = [70, 71, 108, 109, 110, 111, 112]
idx_list_m306_h_45p0_65p0 = [113, 114, 115 ,116]
idx_list_m306_h_50p0_65p0_2 = [154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164]
idx_list_m306_h_50p0_65p0_3 = [230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240] # MN 100000
idx_list_m306_h_45p0_49p5_3 = [241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251] 
idx_list_m306_h_45p0_49p5_4 = [274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284] # MN 40000

# high energy 005_h
idx_list_005_h_45p0_65p0 = [122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132]
idx_list_005_h_45p0_65p0_2 = [133]
idx_list_005_h_50p0_65p0_2 = [175, 176, 177, 178, 179, 180, 193, 216, 217, 218]
idx_list_005_h_50p0_65p0_3 = [219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229] # MN 100000
idx_list_005_h_45p0_49p5_3 = [252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262]
idx_list_005_h_45p0_49p5_4 = [263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273] # MN 40000

# high energy 1200_l
idx_list_1200_l_40p0_70p0 = [69]
idx_list_1200_l_45p0_65p0 = [144, 145, 146, 147, 148, 149, 150, 151, 152, 153]
idx_list_1200_l_40p0_70p0_2 = [95]
idx_list_1200_l_50p0_65p0_2 = [165, 166,167, 168, 169, 170, 171, 172, 173, 174]

# ultra high energy 005_h
idx_list_005_h_65p5_75p0 = [194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204]
idx_list_005_h_65p5_75p0_2 = [219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229]

# ultra high energy 005_h
idx_list_m306_h_65p5_75p0 = [205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215]
idx_list_m306_h_65p5_75p0_2 = [230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240]

# etc 